In [4]:
import cv2
import numpy as np
import torch

In [5]:
#image_data
batch_imgs = [cv2.imread("two_face.jpeg")]

# init_model(ref:faceswap)
input_size = 360
model_filename = ["./resnet_ssd_v1/resnet_ssd_v1.caffemodel",
                  "./resnet_ssd_v1/resnet_ssd_v1.prototxt"]
cv2_dnn_model = cv2.dnn.readNetFromCaffe(model_filename[1], model_filename[0])
cv2_dnn_model.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [6]:
input_data=cv2.dnn.blobFromImages(batch_imgs,
                      scalefactor=1.0,
                      size=(input_size,input_size),
                      mean=[104, 117, 123],
                      swapRB=False,
                      crop=False)
# predict
cv2_dnn_model.setInput(input_data)
predictions = cv2_dnn_model.forward()
# predictions.shape -> (1,1,200,7)

In [7]:
#ground_truth
ground_truth_box=[]
thresh_truth_confidence=0.9
for i in range(predictions.shape[2]):
    #批次大小，通道数（任务），输出空间个数（框的个数），每个框的属性个数
    #属性：2-conf 3~6-rect 0,1-[0.0,1.0]??
    confidence = predictions[0, 0, i, 2]
    if confidence >= thresh_truth_confidence:
        ground_truth_box.append([(predictions[0, 0, i, 3]), 
                                 (predictions[0, 0, i, 4]), 
                                 (predictions[0, 0, i, 5]), 
                                 (predictions[0, 0, i, 6])])
    else:
        break

In [8]:
pred_box = []
list_conf=[]
thresh_confidence=0.01 #如果是0，log(conf)出问题

for i in range(predictions.shape[2]):
    #批次大小，通道数（任务），输出空间个数（框的个数），每个框的属性个数
    #属性：2-conf 3~6-rect 0,1-[0.0,1.0]??
    confidence = predictions[0, 0, i, 2]
    if confidence >= thresh_confidence:
        list_conf.append(predictions[0,0,i,2])
        pred_box.append([(predictions[0, 0, i, 3]), #* input_size
                      (predictions[0, 0, i, 4]),    #* input_size
                      (predictions[0, 0, i, 5]),    #* input_size
                      (predictions[0, 0, i, 6])])   #* input_size
    else:
        break

In [9]:
def calculate_iou(box1, box2):
    x_left = max(box1[0], box2[0])
    y_top = max(box1[1], box2[1])
    x_right = min(box1[2], box2[2])
    y_bottom = min(box1[3], box2[3])

    # 判断是否有重叠区域
    if x_right < x_left or y_bottom < y_top:
        return 0.0  # 没有交集

    # 交集面积
    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    # 并集面积
    union_area = box1_area + box2_area - intersection_area
    iou = intersection_area / union_area
    return iou

In [10]:
#IOU 阈值
thresh_IOU = 0.3
potential_true_index = []
potential_false_index = []

for i,pred in enumerate(pred_box):    
    math_found=False
    for truth in ground_truth_box:
        if calculate_iou(pred,truth)>thresh_IOU:
            potential_true_index.append(i)
            math_found=True
            break
    if not math_found:
        potential_false_index.append(i)
print(potential_true_index)
print(potential_false_index)

[0, 1]
[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104]


In [11]:
def loss_function(list_conf,potential_true_index,potential_false_index,threshold_false_box=1000):
    true_detection_loss=0
    false_detection_loss=0
    for i in potential_true_index:
        true_detection_loss+=np.log(1-list_conf[i])
    #threshold_false_box 防止false_box过多
    for j in potential_false_index[:threshold_false_box]:
        false_detection_loss+=np.log(list_conf[j])
    loss=true_detection_loss+false_detection_loss
    return loss

In [12]:
# print(loss_function(list_conf,potential_true_index,potential_false_index))

In [13]:
def pgd_attack(model, input_data, list_conf, potential_true_index, potential_false_index,
               eps=0.03, alpha=0.01, attack_steps=10, device='cpu'):
    # 将输入数据转为 PyTorch 张量，并确保可以计算梯度
    input_data = torch.tensor(input_data, requires_grad=True, device=device)
    input_data_orig = input_data.clone().detach()  # 原始输入的副本

    for step in range(attack_steps):
        # 前向传播并计算损失
        model.setInput(input_data.detach().cpu().numpy())  # 将输入送入模型
        predictions = model.forward()  # 获取模型预测
        predictions = np.squeeze(predictions)  # 去掉冗余维度

        # 计算损失
        loss = loss_function(
            list_conf, potential_true_index, potential_false_index)
        loss = torch.tensor(loss, requires_grad=True, device=device)

        # 反向传播，计算梯度
        loss.backward()

        # 梯度上升更新（针对 untargeted attack）
        grad = input_data.grad.data
        input_data = input_data + alpha * grad.sign()

        # 投影步骤，将输入限制在 (input_data_orig - eps) 到 (input_data_orig + eps) 范围内
        input_data = torch.clamp(
            input_data, input_data_orig - eps, input_data_orig + eps)

        # 重新约束输入到合法范围 (0, 255) 范围
        input_data = torch.clamp(input_data, 0, 255)

        # 清空梯度，以便下一步计算
        input_data.grad.zero_()

    # 返回对抗样本
    return input_data.detach().cpu().numpy()

In [15]:
cv2_dnn_model

< cv2.dnn.Net 0000018B3D9CBCB0>

In [14]:
pgd_attack(cv2_dnn_model,input_data,list_conf,potential_true_index,potential_false_index)
    

AttributeError: 'NoneType' object has no attribute 'data'

In [ ]:
#在原图画pred_box
for img in batch_imgs:
    for (x1,y1,x2,y2) in pred_box:
        height,width,_=img.shape
        x1, y1, x2, y2 = int(x1*width), int(y1*height), int(x2*width), int(y2*height)
        color = (0, 255, 0)
        thickness = 2  
        cv2.rectangle(img, (x1, y1), (x2, y2), color, thickness)

    # 显示图像
    cv2.imshow("Image with Faces", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()